# Dự đoán tình trạng xe với Logistic Regression

In [52]:
import numpy as np
import scipy.stats as stats
from matplotlib import pyplot as plt

# 1. Load và convert dữ liệu 

In [53]:
# Header: #price,maintenance,doors,persons,lug_capacity,safety,output (the condition of the car)
raw_data = np.genfromtxt('data/used_car_evaluation.csv', dtype=str, 
                         delimiter=',', skip_header=1)
print(raw_data[0])
print(raw_data[-1])
print(raw_data.shape)

['vhigh' 'vhigh' '2' '2' 'small' 'low' 'unacc']
['low' 'low' '5more' 'more' 'big' 'high' 'vgood']
(1728, 7)


In [54]:
# Kiểm tra các giá trị duy nhất của từng cột dữ liệu
# Features: 6 cột đầu
# Giá trị cần đoán: cột thứ 7 
#    (tình trạng xe: acceptable, good, unacceptable, very-good)
print(np.unique(raw_data[:,0]))
print(np.unique(raw_data[:,1]))
print(np.unique(raw_data[:,2]))
print(np.unique(raw_data[:,3]))
print(np.unique(raw_data[:,4]))
print(np.unique(raw_data[:,5]))
print(np.unique(raw_data[:,6]))

['high' 'low' 'med' 'vhigh']
['high' 'low' 'med' 'vhigh']
['2' '3' '4' '5more']
['2' '4' 'more']
['big' 'med' 'small']
['high' 'low' 'med']
['acc' 'good' 'unacc' 'vgood']


**Vì dữ liệu là dạng categorical, cần chuyển sang dạng số**

In [55]:
# Sử dụng LabelEncoder của thư viện sklearn để chuyển từng cột sang dạng số
# Lưu ý: phải thực hiện trên từng cột, vì mỗi cột là 1 feature

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
LE = LabelEncoder()

X = np.stack([LE.fit_transform(col) for col in raw_data.T[0:6]], axis=1)
print(X.shape)

Y = LE.fit_transform(raw_data[:,6])
print(Y.shape)

(1728, 6)
(1728,)


In [56]:
print(X[0])
print(Y[0])

[3 3 0 0 2 1]
2


# 2. Chia dữ liệu ra thành 2 tập train và test

In [57]:
np.random.seed(3000)
rand_indices = np.arange(1728)
np.random.shuffle(rand_indices)
print(rand_indices)

[1296 1391  123 ... 1016 1587 1532]


In [58]:
N_TRAIN = 1600
train_indices = rand_indices[0:N_TRAIN]
test_indices = rand_indices[N_TRAIN:]

x_train, y_train = X[train_indices], Y[train_indices]
print(x_train.shape, y_train.shape)

x_test, y_test = X[test_indices], Y[test_indices]
print(x_test.shape, y_test.shape)

(1600, 6) (1600,)
(128, 6) (128,)


In [59]:
print(x_test[0:6])
print(y_test[0:6])

[[2 3 3 1 2 2]
 [1 3 3 1 0 2]
 [3 0 3 1 0 1]
 [1 2 0 0 1 0]
 [1 2 0 0 0 0]
 [2 0 1 1 1 2]]
[2 0 2 2 2 2]


# 3. kNN

## 3.1. Hàm tính khoảng cách theo công thức Euclidian

In [ ]:
def tinh_khoang_cach(test_s, train_s):
    kc = np.sqrt(np.sum((test_s - train_s)**2))
    return kc

In [60]:
# Tính thử một trường hợp
print(x_test[0], x_train[0])
tinh_khoang_cach(x_test[0], x_train[0])

[2 3 3 1 2 2] [1 3 0 0 2 1]


3.4641016151377544

## 3.2. Thử dùng kNN để phân lớp sample đầu tiên

**Tìm các kNNs cho sample đầu tiên trong tập test**

In [ ]:
K = 13
test_sample = x_test[0]

list_kc = []
for train_sample in x_train:
    kc_i = tinh_khoang_cach(test_sample, train_sample)
    list_kc.append(kc_i)

In [62]:
# Lấy ra list chỉ gồm K phần tử có khoảng cách gần nhất
list_kc = np.array(list_kc)
list_kNN_index = np.argsort(list_kc)[0:K]

**Kiểm tra lại các khoảng cách gần nhất**

In [63]:
print(list_kc[list_kNN_index])

[1.         1.         1.         1.         1.         1.
 1.         1.         1.41421356 1.41421356 1.41421356 1.41421356
 1.41421356]


**Kiểm tra lại các sample gần nhất**

In [65]:
print(x_test[0])
x_train[list_kNN_index]

[2 3 3 1 2 2]


array([[3, 3, 3, 1, 2, 2],
       [2, 3, 2, 1, 2, 2],
       [2, 2, 3, 1, 2, 2],
       [2, 3, 3, 0, 2, 2],
       [2, 3, 3, 1, 1, 2],
       [2, 3, 3, 2, 2, 2],
       [1, 3, 3, 1, 2, 2],
       [2, 3, 3, 1, 2, 1],
       [3, 2, 3, 1, 2, 2],
       [3, 3, 3, 2, 2, 2],
       [2, 2, 2, 1, 2, 2],
       [2, 3, 2, 1, 2, 1],
       [2, 2, 3, 0, 2, 2]])

**Tìm nhãn của kNN điểm trong tập train**

In [67]:
y_train[list_kNN_index]

array([2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2])

**Tìm "vote" cho nhãn của test sample**

In [68]:
np.unique(y_train[list_kNN_index], return_counts=True)

(array([0, 2]), array([ 3, 10]))

**Kiểm tra lại nhãn thật**

In [69]:
y_test[0]

2